In [ ]:
spark

# Regresión

## Generación de datos

Definimos los datos que vamos a tratar de estimar. Vamos a usar el dataset [Friedman1](http://scikit-learn.org/stable/modules/generated/sklearn.datasets.make_friedman1.html), un dataset sintético disponible en scikit-learn. Se genera mediante la fórmula:

$$ F_1 = sin(\pi X_1 X_2) + 20\dot (X_3-1/2)^2 + 10X_4 + 5X_5 + k \sigma(0,1) $$


donde $X_i$ son variables aleatorias uniformes definidas en diferentes intervalos, y $\sigma$ is ruido Gaussiano.

Por tanto, nuestro objetivo es ser capaces de producir $F_1$ a partir de las _features_ dadas por $X_i$. A todos los efectos, nuestro algoritmo de regresión está intentando "aprender" la función $F_1$ que relaciona las entradas con la salida. Nótese que el aprendizaje perfecto nunca será posible, ya que la señal incluye una componente de ruido que no es posible "aprender", ya que no depende de nuestras _features_



In [ ]:
from pyspark.sql import functions as F

In [ ]:
df = spark.range(0, 10000, numPartitions=6)
dfX = df.select("id", 
                F.rand(seed=100).alias("X1"), 
                F.rand(seed=795).alias("X2"), 
                F.rand(seed=231).alias("X3"), 
                F.rand(seed=391).alias("X4"), 
                F.rand(seed=469).alias("X5"), 
                F.randn(seed=27).alias("n"))

In [ ]:
dfX.rdd.getNumPartitions()

In [ ]:
from math import pi
dfY = dfX.withColumn( "Y",  F.sin(pi*dfX.X1*dfX.X2) + 20*(dfX.X3-1/2)**2 + 10*dfX.X4 + 5*dfX.X5 + dfX.n )

In [ ]:
dfY.limit(6).toPandas()

Vamos a pintar la variable dependiente

In [ ]:
%matplotlib inline

import matplotlib.pyplot as plt

plt.style.use('fivethirtyeight')
plt.rcParams['figure.figsize'] = 16, 6;
plt.rcParams['lines.linewidth'] = 1.5

In [ ]:
Y = dfY.select('Y').collect()
plt.figure( 1, (16,5), dpi=200)
p1 = plt.plot( Y )

## Transformación
Convertimos los datos de entrada al formato que necesita Spark (todos los valores en una columna)

In [ ]:
from pyspark.ml.feature import VectorAssembler

assembler = VectorAssembler(inputCols=["X1", "X2", "X3", "X4", "X5"], outputCol="features")

dfF = assembler.transform(dfY)

In [ ]:
dfF.limit(10).toPandas()

## Regresión

In [ ]:
from pyspark.ml.regression import GBTRegressor

reg = GBTRegressor(labelCol='Y', maxIter=20, maxDepth=5, seed=42)
model = reg.fit(dfF)

In [ ]:
dfR = model.transform(dfF)

In [ ]:
result = dfR.select(['Y','prediction']).toPandas()

In [ ]:
fig, ax = plt.subplots(figsize=(14,8))

result.plot(y='Y', ax=ax)
result.plot(y='prediction', ax=ax);
ax.set_xlim(0, 100);

## Evaluación

Para hacer una evaluación real, dividimos en entrenamiento y test

In [ ]:
dfTrain, dfTest = dfF.randomSplit([0.8, 0.2], seed=40)

In [ ]:
model = reg.fit(dfTrain)

In [ ]:
from pyspark.sql import functions as F
from math import sqrt

def rmse(df):
    '''cálculo del error cuadrático medio'''
    err = df.select( F.sum((df.Y - df.prediction)**2) )
    return sqrt(err.take(1)[0][0] / df.count())

In [ ]:
dfTrainResult = model.transform(dfTrain)
print( "TRAIN =", rmse(dfTrainResult))

In [ ]:
dfTestResult = model.transform(dfTest)
print( "TEST =", rmse(dfTestResult))